In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df_biz = pd.read_csv('yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.csv')

In [9]:
df_rev = pd.read_csv('yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.csv')

In [6]:
df_biz.shape

(77445, 98)

In [10]:
df_rev.shape

(2225213, 10)

In [15]:
df_rev.iloc[2].text

'Yes this place is a little out dated and not opened on the weekend. But other than that the staff is always pleasant and fast to make your order. Which is always spot on fresh veggies on their hoggies and other food. They also have daily specials and ice cream which is really good. I had a banana split they piled the toppings on. They win pennysaver awards ever years i see why.'

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
cnx = create_engine('postgresql://username:password@ip_address:port/dbname')